<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="right">


# Procesamiento de lenguaje natural


## TP2

### Alumno: Emmanuel Cardozo





## Ejercicio
Tomar un ejemplo de los bots utilizados y construir el propio.


# Objetivos 

Para la resolución del desafío se generará un Bot con Spacy utilizando un corpus extraído de wikipedia .
Se trabajará con los módulos Spacy y NTLK, barriendo los siguientes ítems:
    
* Armado del corpus    
* Tokenización
* Remoción de caracteres especiales y tags HTML
* Tratamiento de signos de puntuación 
* Lematización del texto
* Remoción de stop words


## 1- instalamos e importamos las librerias necesarias

In [4]:
!pip install spacy==3.3 --quiet
!pip install spacy-stanza==1.0.0 --quiet

In [17]:
import json
import string
import random
import re
import unicodedata
import urllib.request

import nltk
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import bs4 as bs
import stanza
import spacy_stanza

In [6]:
# Descargamos el diccionario en español y armar el pipeline de NLP con spacy
stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")


INFO:stanza:Downloading default packages for language: es (Spanish)...
INFO:stanza:File exists: /root/stanza_resources/es/default.zip.
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


# Cargamos el corpus que va a utilizar el  bot
El corpus será un artículo de wikipedia sobre el escritor argentino Jorge Luis Borges.

URL: https://es.wikipedia.org/wiki/Jorge_Luis_Borges


In [32]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Jorge_Luis_Borges')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

### Texto completo todo en minúsculas

In [33]:
article_text

"\njorge francisco isidoro luis borges (buenos aires, 24 de agosto de 1899 -ginebra, 14 de junio de 1986), más conocido como jorge luis borges, fue un destacado escritor de cuentos, poemas y ensayos argentino, extensamente considerado una figura clave tanto para la literatura en habla hispana como para la literatura universal.[1]\u200b también fue bibliotecario, profesor, conferencista y traductor. sus dos libros más conocidos, ficciones y el aleph, publicados en los años cuarenta, son recopilaciones de cuentos conectados por temas comunes de forma fantástica, como los sueños, los laberintos, las bibliotecas, los espejos, los autores ficticios y las mitologías europeas (como la griega y la nórdica), con argumentos que exploran ideas filosóficas relacionadas, por ejemplo, con la memoria, la eternidad, la posmodernidad y la metaficción.[2]\u200b las obras de borges han contribuido ampliamente a la literatura filosófica, al género fantástico y al posestructuralismo. según marcan numerosos

In [34]:
print("Cantidad de caracteres en el artículo es:", len(article_text))

Cantidad de caracteres en el artículo es: 81693


In [35]:
lista_caracteres = list(set(article_text))
print("Lista de caracteres del texto: ",lista_caracteres)

Lista de caracteres del texto:  ['ú', '6', 'e', 'ñ', '}', ']', 'p', 'º', 'è', 'b', '1', 'j', 'á', 'í', 'i', ';', '?', 'a', 's', 'f', 'ó', 'k', 'g', 'l', '2', '“', '8', 'd', 'r', 'v', 'u', '>', '#', 'y', '0', 'm', '/', 'ö', 'h', ',', '\u200b', '”', '‘', 'é', 'z', '¿', '\n', '4', '{', '.', '@', '7', '9', 'o', '\xa0', 'q', 'ê', '-', '’', "'", '(', ':', '[', '—', 'w', 't', 'c', '5', '…', 'x', '3', 'ü', '%', '«', 'n', ' ', '–', ')', '»']


##### Preprocesamiento del texto
- Removemos caracteres especiales y acentos
- Removemos espacios y saltos de línea
- Removeos las referencias bibliográficas

##### Función de limpieza

In [37]:
def preprocess_clean_text(text):    
    # sacar tildes de las palabras
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # auitar los suscriptis
    text = re.sub("[\[].*?[\]]", "", text)
    text = text.replace('\u200b', '')
    
    # quitar caracteres especiales
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    return text


### Aplicamos la limpieza y vemos el texto resultante

In [38]:
text = preprocess_clean_text(article_text)

In [39]:
text

" jorge francisco isidoro luis borges (buenos aires, 24 de agosto de 1899 -ginebra, 14 de junio de 1986), mas conocido como jorge luis borges, fue un destacado escritor de cuentos, poemas y ensayos argentino, extensamente considerado una figura clave tanto para la literatura en habla hispana como para la literatura universal. tambien fue bibliotecario, profesor, conferencista y traductor. sus dos libros mas conocidos, ficciones y el aleph, publicados en los anos cuarenta, son recopilaciones de cuentos conectados por temas comunes de forma fantastica, como los suenos, los laberintos, las bibliotecas, los espejos, los autores ficticios y las mitologias europeas (como la griega y la nordica), con argumentos que exploran ideas filosoficas relacionadas, por ejemplo, con la memoria, la eternidad, la posmodernidad y la metaficcion. las obras de borges han contribuido ampliamente a la literatura filosofica, al genero fantastico y al posestructuralismo. segun marcan numerosos criticos, el comie

In [40]:
print("Cantidad de caracteres en el artículo es:", len(text))

Cantidad de caracteres en el artículo es: 80479


In [41]:
lista_caracteres = list(set(text))
print("Lista de caracteres del texto: ",lista_caracteres)

Lista de caracteres del texto:  ['2', '8', '6', '4', '(', 'd', '{', 'e', '.', ':', '@', '}', 'r', 'v', '7', 'w', 't', 'c', 'p', 'u', '>', '#', '9', 'y', 'b', '0', '5', 'o', '1', 'j', 'm', 'x', '/', 'q', 'i', 'h', '3', ',', '%', ';', 'g', '?', 'n', "'", ' ', 'a', '-', 'z', 's', ')', 'f', 'k', 'l']


In [42]:
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords

In [43]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
# Generamos el corpus para ver el listado de documentos generados a partir de nuestro artículo
corpus = nltk.sent_tokenize(text)
corpus

[' jorge francisco isidoro luis borges (buenos aires, 24 de agosto de 1899 -ginebra, 14 de junio de 1986), mas conocido como jorge luis borges, fue un destacado escritor de cuentos, poemas y ensayos argentino, extensamente considerado una figura clave tanto para la literatura en habla hispana como para la literatura universal.',
 'tambien fue bibliotecario, profesor, conferencista y traductor.',
 'sus dos libros mas conocidos, ficciones y el aleph, publicados en los anos cuarenta, son recopilaciones de cuentos conectados por temas comunes de forma fantastica, como los suenos, los laberintos, las bibliotecas, los espejos, los autores ficticios y las mitologias europeas (como la griega y la nordica), con argumentos que exploran ideas filosoficas relacionadas, por ejemplo, con la memoria, la eternidad, la posmodernidad y la metaficcion.',
 'las obras de borges han contribuido ampliamente a la literatura filosofica, al genero fantastico y al posestructuralismo.',
 'segun marcan numerosos c

##### Función de procesamiento del texto utilizando Spacy
- Tokenización
- Lemmatización 
- Remoción de stopwords 
- Remoción de signos de puntuación

In [45]:
def get_processed_text(text):
    doc = nlp(text)
    
    # Tokenization y lemmatization
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    
    # Filtrado de stop words
    filtered_sentence =[]
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    
    # Filtrado de puntuacion
    puntuacion = string.punctuation + "¿¡"
    filtered_sentence = [w for w in filtered_sentence if w not in puntuacion]

    return filtered_sentence

In [46]:
filtered = get_processed_text(text)

In [47]:
print("Palabras resultantes del procesamiento:\n",filtered)

Palabras resultantes del procesamiento:
 ['jorge', 'francisco', 'isidoro', 'luis', 'borges', 'aire', '24', 'agosto', '1899', '-ginebra', '14', 'junio', '1986', 'jorge', 'luis', 'borges', 'destacado', 'escritor', 'cuento', 'poema', 'ensayo', 'argentino', 'extensamente', 'considerado', 'figura', 'clave', 'literatura', 'hispano', 'literatura', 'universal', 'bibliotecario', 'profesor', 'conferencista', 'traductor', 'libro', 'conocido', 'ficción', 'aleph', 'publicado', 'ano', 'cuarenta', 'recopilación', 'cuento', 'conectado', 'tema', 'común', 'forma', 'fantastico', 'sueno', 'laberinto', 'biblioteca', 'espejo', 'autor', 'ficticio', 'mitologia', 'europeo', 'griega', 'nordica', 'argumento', 'explorar', 'idea', 'filosofico', 'relacionado', 'ejemplo', 'memoria', 'eternidad', 'posmodernidad', 'metaficcion', 'obra', 'borge', 'contribuir', 'ampliamente', 'literatura', 'filosofico', 'genero', 'fantastico', 'posestructuralismo', 'marcar', 'numeroso', 'critico', 'comienzo', 'realismo', 'magico', 'lite

In [48]:
from spacy.lang.es.stop_words import STOP_WORDS as STOP_WORDS_spacy
spacy_stopwords = STOP_WORDS_spacy

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    corpus.append(user_input)

    # Creamos un vectorizar TFIDF que quite las "stop words" en epañol y 
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words=spacy_stopwords)

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "Lo siento. No puedo entenderte. Prueba con otra consulta"
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

##### Probamos el bot:

In [50]:
question = "¿Borges ganó el premio Nobel?" 
resp = generate_response(question,corpus)
print("Pregunta: ",question)
print("\nRespuesta: ",resp)

Pregunta:  ¿Borges ganó el premio Nobel?

Respuesta:  con ocasion de otro doctorado honoris causa, en 1976, en el chile de pinochet, recibio una llamada de estocolmo advirtiendole de que si acudia a recogerlo nunca iba a ganar el nobel.


In [51]:
question = "¿Cuando escribió Ficciones?"
resp = generate_response(question,corpus)
print("Pregunta: ",question)
print("\nRespuesta: ",resp)

Pregunta:  ¿Cuando escribió Ficciones?

Respuesta:  aparecio ficciones en italiano, bajo el titulo la biblioteca di babele.


In [52]:
question = "¿Que relación tenia borges con el peronismo?"
resp = generate_response(question,corpus)
print("Pregunta: ",question)
print("\nRespuesta: ",resp)

Pregunta:  ¿Que relación tenia borges con el peronismo?

Respuesta:  segun el, se opuso al peronismo porque era liberticida y de raiz fascista.


In [53]:
question = "¿Que relación hay entre la matemática y literaturad de Borges?"
resp = generate_response(question,corpus)
print("Pregunta: ",question)
print("\nRespuesta: ",resp)

Pregunta:  ¿Que relación hay entre la matemática y literaturad de Borges?

Respuesta:  en un libro reciente, borges 2.0: from text to virtual worlds (borges 2.0: del texto a los mundos virtuales), perla sasson-henry explora las relaciones entre la internet descentralizada de youtube, los blogs y wikipedia y los cuentos de borges, que hacen del lector un participante activo.


In [54]:
question = "¿Qué discipulos directos tuvo Borges?"
resp = generate_response(question,corpus)
print("Pregunta: ",question)
print("\nRespuesta: ",resp)

Pregunta:  ¿Qué discipulos directos tuvo Borges?

Respuesta:  si bien borges no ha tenido discipulos directos pues ello supondria una estetica y una escuela previsibles de las que el mismo descreia hay autores contemporaneos que, de acuerdo con sus criticos, han recibido su influencia de modo directo.


##### Tests del bot utilizando gradio


In [55]:
import sys
!{sys.executable} -m pip install gradio --quiet

     |████████████████████████████████| 6.1 MB 5.2 MB/s 
     |████████████████████████████████| 2.3 MB 45.4 MB/s 
     |████████████████████████████████| 270 kB 47.2 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 212 kB 50.8 MB/s 
     |████████████████████████████████| 57 kB 4.2 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 112 kB 54.5 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 80 kB 9.1 MB/s 
     |████████████████████████████████| 68 kB 6.1 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 856 kB 48.1 MB/s 
     |████████████████████████████████| 4.0 MB 41.0 MB/s 
     |████████████████████████████████| 594 kB 58.6 MB/s 


In [56]:
import gradio as gr

def bot_response(human_text):
    print(human_text)
    return generate_response(human_text.lower(), corpus)

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://11579.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


metafísica
Keyboard interruption in main thread... closing server.


KeyboardInterrupt: ignored

#### Conclusiones


- Es importante conocer el texto  de antemano para lograr un buen pre-procesmiento del documento. Si bien las respuestas han tenido cierta coherencia, algunas partes del texto original hubiesen dado una respuesta más certera a las consultas propuestas.

        
     
